In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import re

In [2]:
url = 'https://puzzles-prod.telegraph.co.uk/plusword/data.json'
r_json = requests.get(url).json()

row = {'date' : r_json['copy']['date-publish-analytics'][:10],
      'puzzle_number' : r_json['meta']['number'],
      'plusword_solution' : r_json['settings']['solution']}

for direction in ['across', 'down']:
    clue_num = 0
    for clue in r_json['cluedata'][direction]:
        clue_num +=1
        row.update({'clue_' + direction + '_' + str(clue_num) :clue})

for answer_num in range(1,6):
    row.update({'answer_' + str(answer_num) :r_json['celldata'][5*(answer_num-1):(5*answer_num)]})

In [3]:
url = 'https://puzzles-prod.telegraph.co.uk/plusword/index.html'
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)
driver.get(url)
element = driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/header/button').click()

yellow = []
green = []
match = 'C\d*'
for table_row in driver.find_elements(By.CLASS_NAME, "row"):
    for cell in table_row.find_elements(By.TAG_NAME, 'td'):
        cell_class = cell.get_attribute("class")
                
        if 'right-letter-wrong-column' in cell_class:
            yellow.append(int(re.search(match, cell_class).group(0).strip('C')) + 1)
            
        if 'right-letter-right-column' in cell_class:
            green.append(int(re.search(match, cell_class).group(0).strip('C')) + 1)
            
row.update({'yellow' : yellow,
            'green' : green})

In [4]:

df = pd.DataFrame.from_dict([row])
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.replace('"', '', regex=True)
df = df.replace("&#039", '', regex=True)
df = df.replace(';', '', regex=True)


In [5]:
df

,date,puzzle_number,plusword_solution,clue_across_1,clue_across_2,clue_across_3,clue_across_4,clue_across_5,clue_down_1,clue_down_2,clue_down_3,clue_down_4,clue_down_5,answer_1,answer_2,answer_3,answer_4,answer_5,yellow,green
0,2023-04-23,336,CHEEK,Knives and ____,Whats happening,"Uncertain, dodgy","____ Sam, the USA",Removes rind from,Brassed off,Sheepish?,See how the land lies,Toll,Eyelid sores,FORKS,EVENT,DICEY,UNCLE,PEELS,"[4, 6, 13, 18, 20, 22]","[8, 14, 23]"


In [6]:
%run mumger.py

2023-04-23 20:33:34.362 INFO    selenium.webdriver.common.selenium_manager: Executing: /home/tomwh/.local/lib/python3.10/site-packages/selenium/webdriver/common/linux/selenium-manager --browser firefox --output json


Empty DataFrame
Columns: [date, puzzle_number, plusword_solution, clue_across_1, clue_across_2, clue_across_3, clue_across_4, clue_across_5, clue_down_1, clue_down_2, clue_down_3, clue_down_4, clue_down_5, answer_1, answer_2, answer_3, answer_4, answer_5, yellow, green]
Index: []
empty_df
Empty DataFrame
Columns: [load_ts, user, time]
Index: []
empty_df
